##This package assumes you have tshark installed
###To learn more about tshark please visit: https://www.wireshark.org/docs/man-pages/tshark.html


Shark Dissector -- 2015 Blackhat Data Science Course

Shark Dissector runs through PCAP streams and writes each individual stream to hard drive.
User can specify fields they're looking for within the stream. This example searches for any stream
which has the word "password" in the content. The content is then displayed to the screen for the analyst
to investigate.

This is quicker then searching through each pcap individually and allows analyst/incident 
responders to work with large sets of data.

Use the file named idslog0.pcap located in the data folder to answer the following questions:

1) Identify which stream contains the password chamomileporkorange

2) Find all streams which contain the hash b94b3aac033a736438761e893883391f66c9d2d1

3) Find all streams which have a password in it. Look for pass or PASS to identify the stream with a password

In [1]:
#Choose File
def fileChooser():
    global filename
    global filepath
    global pcap
    
    from tkinter import Tk, filedialog
    root = Tk()
    root.filename =  filedialog.askopenfilename(initialdir = "C:/",title = "Choose PCAP to Disect",
                                                )
    filename = root.filename.split('/')[-1]
    filepath = '/'.join(root.filename.split('/')[:-1])+'/'
    pcap = filepath+filename
    root.destroy()

fileChooser()

#get current time for timestamp
import datetime
current_time = '_'.join(datetime.datetime.ctime(datetime.datetime.now()).replace(':','.').split(' '))

#create directory
import os
subdirectory = filepath+filename+'_'+current_time
os.mkdir(filepath+filename+'_'+current_time)


In [4]:
import subprocess

#linux
#collect_streams = subprocess.Popen(["/usr/local/bin/tshark", "-r", pcap, "-T", "fields", "-e", "tcp.stream"], stdout=subprocess.PIPE)

#windows
collect_streams = subprocess.Popen(["c:/Program Files/Wireshark/tshark", "-r", pcap, "-T", "fields", "-e", "tcp.stream"], stdout=subprocess.PIPE)

streams = collect_streams.stdout.read().splitlines()
unique_streams = sorted(set([str(s, 'UTF8') for s in streams if len(s) > 0]))
print(str(len(unique_streams)), 'streams identified!')

2900 streams identified!


In [1]:
import re
pass_streams = []
for stream in unique_streams:
    #print('Parsing stream #%s' % stream)
    stream_follower = subprocess.Popen(["tshark", "-r", pcap, "-qz", "follow,tcp,ascii,"+stream], stdout=subprocess.PIPE) #> follow-stream-$i.txt
    #stream_follower = subprocess.Popen(["c:/Program Files/Wireshark/tshark", "-r", pcap, "-qz", "follow,tcp,ascii,"+stream], stdout=subprocess.PIPE) #> follow-stream-$i.txt
    content = stream_follower.stdout.read()
    #Modify regular expression string to find items of interest
    #HASH = re.findall(r'(b94b3aac033a736438761e893883391f66c9d2d1)', str(content))
    PASSWORD = re.findall(r'(GET|get)', str(content))
    if PASSWORD:
        print('[!] Investigating stream #%s' % stream)
        data = content.splitlines()
        for row in data:
            try:
                print(str(row, 'UTF-8'))
            except Exception as e:
                print(e)
        pass_streams.append(stream)
    write_file = open(subdirectory+'/'+filename+'_'+stream, 'wb')
    write_file.write(content)
print('Streams to investigate: %s' % ', '.join(pass_streams))

NameError: name 'unique_streams' is not defined

In [ ]:
print(str(len(unique_streams)), 'streams written to %s' % filepath)